In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import time

#ISOMAP

from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection, neighbors)


In [0]:
pip install git+https://github.com/samueljackson92/coranking.git

  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-7baqj3ux
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-7baqj3ux
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=87036 sha256=3d73dc1f8de1406de737c047c2d82dd6b1f28b65d4d2f398bfec404200042286
  Stored in directory: /tmp/pip-ephem-wheel-cache-bk52w1ib/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


###1. Loading and Processing MNIST

In [0]:
link1='https://drive.google.com/open?id=1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
training_data  = pd.read_csv('train.csv')

1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd


In [0]:
labels  = training_data['label']
training_data = training_data.drop("label",axis=1)

In [0]:
print(training_data.shape)

(42000, 784)


In [0]:
from sklearn.preprocessing import StandardScaler
standardized_data = StandardScaler().fit_transform(training_data)
print(standardized_data.shape)

(42000, 784)


In [0]:
covar_matrix = np.matmul(standardized_data.T , standardized_data)

print ( "The shape of variance matrix = ", covar_matrix.shape)

The shape of variance matrix =  (784, 784)


In [0]:
training_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###Decision Tree on Original Data

In [0]:
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(training_data[0:9999], labels[0:9999], test_size=0.25, random_state=49)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.7984

###ISOMAP

###Decision T on ISO Data

In [0]:
from prettytable import PrettyTable
t_iso_10k = PrettyTable(['DT_score_ISOData', 'Dimensions', 'iso_time','DecisionTree_time' ,'total_time', 'trust_iso','cont_iso' ,'lcmc_iso'])

t_iso_24dim = PrettyTable(['DT_score_Orig','DT_score_ISOData', 'Dimensions', 'iso_time','DecisionTree_time' ,'total_time', 'trust_iso','cont_iso' ,'lcmc_iso'])

def MDS_Decision_Tree (training_data, components,size):
  
  #Dt on training{:size}
  X_train, X_test, y_train, y_test = train_test_split(training_data[:size], labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, y_train)
  clf_time = time.time() - clf_start_time
  score1=clf.score(X_test,y_test)
  score1 = str(round(score1, 4))
  
  
  
  #Reducing to mentioned components
  mds_start_time = time.time()
  training_data_isomap = manifold.Isomap(n_components=components).fit_transform(training_data[:size])
  mds_time = time.time() - mds_start_time
  
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_isomap, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, y_train)
  clf_time = time.time() - clf_start_time
  score=clf.score(X_test,y_test)
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))

  Q = coranking.coranking_matrix(training_data[:size], training_data_isomap)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))

  print(size)
  t_iso_24dim.add_row([score1,score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [0]:
dimensions=784
size=9999
while dimensions >= 2:
  MDS_Decision_Tree (training_data, dimensions,size)
  dimensions=int(dimensions/2)

print(t_iso_10k)

+------------------+------------+----------+-------------------+------------+-----------+----------+----------+
| DT_score_ISOData | Dimensions | iso_time | DecisionTree_time | total_time | trust_iso | cont_iso | lcmc_iso |
+------------------+------------+----------+-------------------+------------+-----------+----------+----------+
|      0.832       |    784     | 345.6507 |       9.9944      |  355.6451  |   0.9738  |  0.9619  |  0.5295  |
|      0.8472      |    392     | 332.8778 |       5.0792      |  337.957   |   0.975   |  0.9667  |  0.5347  |
|      0.8632      |    196     | 330.1872 |       2.5233      |  332.7104  |   0.975   |  0.9695  |  0.5329  |
|      0.8608      |     98     | 328.8001 |       1.2901      |  330.0902  |   0.9747  |  0.971   |  0.5264  |
|      0.8708      |     49     | 328.3898 |       0.6226      |  329.0124  |   0.9741  |  0.9717  |  0.5142  |
|      0.8764      |     24     | 327.805  |       0.302       |  328.107   |   0.9729  |  0.9718  |  0.

In [0]:
dimensions=24
size=1000
while size <= 10000:
  MDS_Decision_Tree (training_data, dimensions,size)
  size=size+1000

print(t_iso_24dim)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
+---------------+------------------+------------+----------+-------------------+------------+-----------+----------+----------+
| DT_score_Orig | DT_score_ISOData | Dimensions | iso_time | DecisionTree_time | total_time | trust_iso | cont_iso | lcmc_iso |
+---------------+------------------+------------+----------+-------------------+------------+-----------+----------+----------+
|      0.48     |       0.54       |     24     |  0.0901  |       0.0033      |   0.0933   |   0.9183  |  0.9071  |  0.5167  |
|     0.6067    |      0.7133      |     24     |  3.297   |       0.0261      |   3.3231   |   0.9554  |  0.9511  |  0.5594  |
|     0.632     |       0.68       |     24     |  2.3502  |       0.0217      |   2.3719   |   0.9529  |  0.9473  |  0.5621  |
|     0.672     |      0.778       |     24     | 10.3192  |       0.0572      |  10.3764   |   0.9617  |  0.9578  |  0.5486  |
|     0.6933    |      0.7853      |     24     | 22.